In [56]:
import pandas as pd
import numpy as np
import os
import pickle


In [58]:
# paths
enrollment_path = "./signature/enrollment"
verification_path = "./signature/verification"

In [60]:
# fonctions 

def load_signature(path):
    """
    Loads a signature TSV file that has no header.
    Adds proper column names.
    """
    col_names = ['t', 'x', 'y', 'pressure', 'penup', 'azimuth', 'inclination']
    df = pd.read_csv(path, sep='\t', header=None, names=col_names)
    return df


def compute_velocity(df):
    """
    Computes velocities vx and vy using differences dx/dt and dy/dt
    """
    t = df["t"].values
    x = df["x"].values
    y = df["y"].values
    dt = np.diff(t)
    dx = np.diff(x)
    dy = np.diff(y)

    # Avoid division by zero
    dt[dt == 0] = 1e-6
    vx = dx / dt
    vy = dy / dt

    # Pad to match original length
    vx = np.concatenate(([0], vx))
    vy = np.concatenate(([0], vy))

    return vx, vy

def normalize_feature(feature):
    """
    Normalize array using z-score
    """
    mean = np.mean(feature)
    std = np.std(feature) + 1e-6
    return (feature - mean) / std

def extract_features(df):
    """
    Compute final features: x, y, vx, vy, pressure
    Normalized per signature
    """
    x = df["x"].values
    y = df["y"].values
    pressure = df["pressure"].values

    vx, vy = compute_velocity(df)

    x_norm = normalize_feature(x)
    y_norm = normalize_feature(y)
    vx_norm = normalize_feature(vx)
    vy_norm = normalize_feature(vy)
    p_norm = normalize_feature(pressure)

    features = np.vstack([x_norm, y_norm, vx_norm, vy_norm, p_norm]).T
    return features

def preprocess_signature(path):
    """
    Load and preprocess a single signature TSV file
    """
    df = load_signature(path)
    features = extract_features(df)
    return features

def load_all_signatures(folder):
    """
    Load and preprocess all TSV signatures in a folder
    Returns a dictionary {filename: features}
    """
    signatures = {}
    for file in os.listdir(folder):
        if file.endswith(".tsv"):
            fullpath = os.path.join(folder, file)
            signatures[file] = preprocess_signature(fullpath)
    return signatures


In [62]:
# Load and preprocess all signatures
# -------------------------------
print("Processing enrollment signatures...")
enrollment_signatures = load_all_signatures(enrollment_path)
print(f"Loaded {len(enrollment_signatures)} enrollment signatures.")

print("Processing verification signatures...")
verification_signatures = load_all_signatures(verification_path)
print(f"Loaded {len(verification_signatures)} verification signatures.")

# Save preprocessed signatures
with open("enrollment_signatures.pkl", "wb") as f:
    pickle.dump(enrollment_signatures, f)

with open("verification_signatures.pkl", "wb") as f:
    pickle.dump(verification_signatures, f)


Processing enrollment signatures...
Loaded 150 enrollment signatures.
Processing verification signatures...
Loaded 1350 verification signatures.


=> features (x, y, vx, vy, pressure) are now computed and normalized for all signatures 